In [1]:
# Module Imports
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
%matplotlib inline

In [2]:
# IMDB
imdb_names = pd.read_csv('data/imdb.name.basics.csv.gz',)
imdb_title = pd.read_csv('data/imdb.title.basics.csv.gz',)
# imdb_akas = pd.read_csv('data/imdb.title.akas.csv.gz',)
# imdb_principals = pd.read_csv('data/imdb.title.principals.csv.gz',)
# imdb_ratings = pd.read_csv('data/imdb.title.ratings.csv.gz',)
imdb_title.shape

In [43]:
'''bom_gross = pd.read_csv('data/bom.movie_gross.csv.gz',)
bom_gross.shape'''

In [38]:
'''
# Rotten Tomato
rt_movies = pd.read_csv('data/rt.movie_info.tsv.gz',delimiter='\t')
rt_reviews = pd.read_csv('data/rt.reviews.tsv.gz',delimiter='\t', encoding='latin-1')
'''

In [3]:
# The Movie DB
tmdb_movies = pd.read_csv('data/tmdb.movies.csv.gz',)

In [4]:
tn_budget = pd.read_csv('data/tn.movie_budgets.csv.gz', )
tn_budget2 = pd.read_csv('data/themovies.csv', )
tn_budget = tn_budget.rename(columns={'movie': 'title'})

tn_budget.shape


In [5]:
# Clean the currency strings
def currency_str(x):
    return x.replace(',','').replace('$','')

In [6]:
tn_budget.domestic_gross = tn_budget.domestic_gross.apply(currency_str).astype('float')
tn_budget.worldwide_gross = tn_budget.worldwide_gross.apply(currency_str).astype('float')
tn_budget.production_budget = tn_budget.production_budget.apply(currency_str).astype('float')

In [7]:
tn_budget = tn_budget.where(tn_budget.worldwide_gross>0).dropna()

In [8]:
tn_budget['profit'] = tn_budget.worldwide_gross - tn_budget.production_budget
tn_budget.describe()


### Exploring the Profitability of Genre 
IMDB dataset contains 146144 movie titles.

In [10]:
# Join the TN Profits and IMDB Movie Titles datasets
imdb_title = imdb_title.rename(columns={'original_title':'title','start_year': 'year'})


In [13]:
tn_budget['year'] = pd.to_datetime(tn_budget.release_date).dt.year

In [14]:
tn_imdb_join = imdb_title.join(tn_budget.set_index(['title','year']), on=['title','year'], how='inner')
# drop the empty genres
tn_imdb_join = tn_imdb_join.dropna(subset='genres')
tn_imdb_join.shape


In [16]:
Genres = MultiLabelBinarizer()
Genres.fit(tn_imdb_join.genres.dropna().apply(lambda x: x.split(',')))

In [17]:
imdb_genres = Genres.transform(tn_imdb_join.genres.dropna().apply(lambda x: x.split(',')))

In [19]:
imdb_genres.sum(axis=0)